In [1]:
!pip install -q faker
from faker import Faker

fake = Faker(locale = 'en_US')
fp = fake.simple_profile()
print(fp)
fp['birthdate'] = str(fp['birthdate'])
print(fp)

{'username': 'michael32', 'name': 'Sarah Smith', 'sex': 'F', 'address': '062 Barber Port\nWilliamshire, MT 02296', 'mail': 'devinward@hotmail.com', 'birthdate': datetime.date(1983, 8, 28)}
{'username': 'michael32', 'name': 'Sarah Smith', 'sex': 'F', 'address': '062 Barber Port\nWilliamshire, MT 02296', 'mail': 'devinward@hotmail.com', 'birthdate': '1983-08-28'}


In [2]:

# source: https://github.com/karmus89/blockchain-a-z

import datetime
import hashlib
import json
import sys


class Blockchain:

    def __init__(self):
        
        self.chain = []
        self.create_block(proof=1, previous_hash='0',record = {'username': '', 'name': 'Chelsea ', 'sex': '', 'address': '', 'mail': '', 'birthdate': ''})

    def create_block(self, proof, previous_hash,record):
        "Create a block."
        
        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'record': record
        }
        self.chain.append(block)
        
        return block

    def get_previous_block(self):
        
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

        return True

In [3]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple


class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}'
        
        self.app = Flask(__name__)
        self.add_api_endpoints()
        
        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()
        
    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."
        
        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST','PUT']
        )
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )

    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':
          # In addition to all existing fields (index, timestamp, proof and previous_hash), you will add add a new field named record. Note you will populate the record field using automatically generated person’s profile by python library called faker
            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']
            fp = fake.simple_profile()
            fp['birthdate'] = str(fp['birthdate'])
            record = fp

            proof = self.chain.proof_of_work(prev_proof)

            block = self.chain.create_block(proof, prev_hash,record)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)

        if request.method == 'PUT':
          fp = fake.simple_profile()
          fp['birthdate'] = str(fp['birthdate'])
          modify = self.chain.chain[2]
          modify['record'] = fp
          self.chain.chain[2] = modify

          response = {'message': 'Tried to Modify the 3rd record'}

          return (jsonify(response), 200)

    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        # depricated
        # request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

In [4]:
import time

with BlockchainApp() as blockchain_app:

    # wait for server thread ready
    time.sleep(5)
    #Request to mine 20 blocks. Then, print the resulting blockchain
    for _ in range(20):

        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print("Proof={}".format(response.json()['block']['proof']))

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))
    resultant_block_chain = json.dumps(response.json())
    print()
    print()
    #Find names of all males from the blockchain and print them
    print("All Male names")
    for each in json.loads(resultant_block_chain)['chain']:
      if each['record']['sex'] == 'M':
        print(each['record']['name'])
    print()
    print()
    #Find usernames of all females from the blockchain and print them.
    print("All female usernames")
    for each in json.loads(resultant_block_chain)['chain']:
      if each['record']['sex'] == 'F':
          print(each['record']['username'])
    print()
    print()
    print("Validation")
    #Request to validate the blockchain. It should succeed.
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))
    #Create a new profile record using faker library and directly access blockchain_app object to overwrite 3rd block’s record field with it.
    original = blockchain_app.chain.chain[2]['record']
    headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}
    print()
    print()
    print("Overwrite 3rd block’s record field by directly accessing blockchain_app")
    response = requests.put(f'{blockchain_app.host_url}/blocks', headers=headers)
    print(json.dumps(response.json(), indent=2))


    for i in enumerate(blockchain_app.chain.chain):
      print(i)

    # original = blockchain_app.chain.chain[1]['proof']
    # blockchain_app.chain.chain[1]['proof'] = 12345
    print()
    print()
    print('Records: Original={}, Forged={}'.format(original,blockchain_app.chain.chain[2]['record']))
    
    # response = requests.get(f'{blockchain_app.host_url}/blocks')
    # print(json.dumps(response.json(), indent=2))
    #Request to validate the blockchain again. It should fail since the blockchain has been altered
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))
    
    # proof_1 = blockchain_app.chain.chain[1]['record']
    # proof_2 = blockchain_app.chain.chain[2]['proof']

    # forged_hash = blockchain_app.chain.hash_proof(proof_1, proof_2)

    # print("Hashes: \n\tOriginal =\t{}\n\tForged =\t{}".format(blockchain_app.chain.chain[2]['previous_hash'],forged_hash))


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:26] "POST /blocks HTTP/1.1" 200 -


Proof=533


INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:27] "POST /blocks HTTP/1.1" 200 -


Proof=45293


INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:27] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:27] "POST /blocks HTTP/1.1" 200 -


...

Proof=107780
Proof=47346


INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:29] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "POST /blocks HTTP/1.1" 200 -


Proof=46891


INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "GET /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "GET /blocks/validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "PUT /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "GET /blocks/validate HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2023 02:37:30] "GET /shutdown HTTP/1.1" 200 -


Proof=91004
Chain={
  "chain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "record": {
        "address": "",
        "birthdate": "",
        "mail": "",
        "name": "Chelsea ",
        "sex": "",
        "username": ""
      },
      "timestamp": "2023-04-15 02:37:21.431492"
    },
    {
      "index": 1,
      "previous_hash": "b4f5e0ad27770e6be8fa0df9724995eb093908ec201bf9c4ff0af4b68f59794d",
      "proof": 533,
      "record": {
        "address": "389 Walls Ramp Suite 495\nMeganport, KS 44551",
        "birthdate": "1910-08-18",
        "mail": "cdelacruz@hotmail.com",
        "name": "Leslie Hudson",
        "sex": "F",
        "username": "angela14"
      },
      "timestamp": "2023-04-15 02:37:26.460060"
    },
    {
      "index": 2,
      "previous_hash": "b6907b45fe11052853f949d8620fc4321ad81869bc6655c096ee1c3a9e5a90c3",
      "proof": 45293,
      "record": {
        "address": "PSC 4373, Box 5314\nAPO AA 42500",
        "birthdate": "

In [5]:
# headers = {
#     'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}
# print()
# print("Overwrite 3rd block’s record field by directly accessing blockchain_app")
# response = requests.put(f'{blockchain_app.host_url}/blocks', headers=headers)
# print(json.dumps(response.json(), indent=2))